# setup

In [123]:
import re
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import types, Row, Column

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

# check file size

In [3]:
!wc -l data/train.txt

45840617 data/train.txt


In [4]:
!wc -l data/test.txt

6042135 data/test.txt


# experimenting and some EDA with DFs

In [5]:
train = spark.read.option('header', 'false').csv('data/train.txt', sep='\t')

In [6]:
train.write.format('parquet').save('data/train.parquet')
train_parquet = spark.read.parquet('data/train.parquet')

In [9]:
!du data/train.txt
!du data/train.parquet

10885924	data/train.txt
2895045	data/train.parquet


In [7]:
%%time
train.count()

CPU times: user 20 ms, sys: 60 ms, total: 80 ms
Wall time: 2min 47s


45840617

In [8]:
%%time
train_parquet.count()

CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 2.85 s


45840617

In [15]:
test = spark.read.option('header','false').csv('data/test.txt', sep='\t')
test.write.format('parquet').save('data/test.parquet')
del test
# test_parquet = spark.read.parquet('data/test.parquet')

In [12]:
# %%time
# test_parquet.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 327 ms


6042135

In [94]:
train = spark.read.parquet('data/train.parquet')
# s = train.sample(False, 0.05)

In [11]:
# test = spark.read.parquet('data/test.parquet')

In [ ]:
s.printSchema()

In [4]:
# df = spark.read.option('header', 'false').csv('data/toy_train.txt', sep='\t')
# df.head()

In [5]:
# %%time
# df.count()

In [6]:
# df.select(df.columns[2]).show(5)

In [7]:
# # df.write.format('parquet').save('data/toy_train.parquet')
# df = spark.read.parquet('data/toy_train.parquet')
# df.printSchema()

In [9]:
# for c in df.columns:
#     df = df.withColumnRenamed(c, c.strip('_'))
# df.columns[:5]

In [8]:
# for c in df.columns[:14]:
#     df = df.withColumn(c, df[c].cast('float'))
# df.printSchema()

In [95]:
# full training set
for c in train.columns:
    train = train.withColumnRenamed(c, c.strip('_'))
# train.columns[:5]

In [4]:
# sample
for c in s.columns:
    s = s.withColumnRenamed(c, c.strip('_'))
# s.columns[:5]

In [96]:
# full training set
for c in train.columns[:14]:
    train = train.withColumn(c, train[c].cast('float'))
# train.printSchema()

In [5]:
# sample
for c in s.columns[:14]:
    s = s.withColumn(c, s[c].cast('float'))
# s.printSchema()

In [98]:
%%time
# full training set
stats = train[train.columns[1:14]].describe()
means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]
stdevs = np.array(stats[stats['summary'] == 'stddev'].collect())[0][1:]

CPU times: user 70 ms, sys: 30 ms, total: 100 ms
Wall time: 1min 59s


In [12]:
%%time
# sample
stats = s[s.columns[1:14]].describe()
means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]
stdevs = np.array(stats[stats['summary'] == 'stddev'].collect())[0][1:]

CPU times: user 20 ms, sys: 60 ms, total: 80 ms
Wall time: 25 s


In [99]:
stats.show(vertical=True)

-RECORD 0---------------------
 summary | count              
 c1      | 25047061           
 c2      | 45840617           
 c3      | 36001170           
 c4      | 35903248           
 c5      | 44657500           
 c6      | 35588289           
 c7      | 43857751           
 c8      | 45817844           
 c9      | 43857751           
 c10     | 25047061           
 c11     | 43857751           
 c12     | 10768965           
 c13     | 35903248           
-RECORD 1---------------------
 summary | mean               
 c1      | 3.5024133170754044 
 c2      | 105.84841979766546 
 c3      | 26.913041020611274 
 c4      | 7.322680248873305  
 c5      | 18538.991664871523 
 c6      | 116.06185085211598 
 c7      | 16.333130032135028 
 c8      | 12.517042137556713 
 c9      | 106.1098234380509  
 c10     | 0.6175294977722137 
 c11     | 2.7328343170173044 
 c12     | 0.9910356287721244 
 c13     | 8.217461161174054  
-RECORD 2---------------------
 summary | stddev             
 c1     

In [101]:
stats2 = train[train.columns[14:]].summary('count')

In [102]:
stats2.show(vertical=True)

-RECORD 0-----------
 summary | count    
 c14     | 45840617 
 c15     | 45840617 
 c16     | 44281144 
 c17     | 44281144 
 c18     | 45840617 
 c19     | 40299992 
 c20     | 45840617 
 c21     | 45840617 
 c22     | 45840617 
 c23     | 45840617 
 c24     | 45840617 
 c25     | 44281144 
 c26     | 45840617 
 c27     | 45840617 
 c28     | 45840617 
 c29     | 44281144 
 c30     | 45840617 
 c31     | 45840617 
 c32     | 25667759 
 c33     | 25667759 
 c34     | 44281144 
 c35     | 10885544 
 c36     | 45840617 
 c37     | 44281144 
 c38     | 25667759 
 c39     | 25667759 



In [109]:
N = train.count()
N

45840617

In [114]:
nans = {}
for c in stats2.columns[1:]:
    nans[c] = N - int(stats2.select(c).collect()[0][0])
nans

{'c14': 0,
 'c15': 0,
 'c16': 1559473,
 'c17': 1559473,
 'c18': 0,
 'c19': 5540625,
 'c20': 0,
 'c21': 0,
 'c22': 0,
 'c23': 0,
 'c24': 0,
 'c25': 1559473,
 'c26': 0,
 'c27': 0,
 'c28': 0,
 'c29': 1559473,
 'c30': 0,
 'c31': 0,
 'c32': 20172858,
 'c33': 20172858,
 'c34': 1559473,
 'c35': 34955073,
 'c36': 0,
 'c37': 1559473,
 'c38': 20172858,
 'c39': 20172858}

In [120]:
newRow = spark.createDataFrame(Row(nans))
newRow.show()

/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+
|c14|c15|    c16|    c17|c18|    c19|c20|c21|c22|c23|c24|    c25|c26|c27|c28|    c29|c30|c31|     c32|     c33|    c34|     c35|c36|    c37|     c38|     c39|
+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+
|  0|  0|1559473|1559473|  0|5540625|  0|  0|  0|  0|  0|1559473|  0|  0|  0|1559473|  0|  0|20172858|20172858|1559473|34955073|  0|1559473|20172858|20172858|
+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+



In [125]:
newRow = newRow.withColumn('summary', F.lit('nan_count'))
newRow.show()

+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+---------+
|c14|c15|    c16|    c17|c18|    c19|c20|c21|c22|c23|c24|    c25|c26|c27|c28|    c29|c30|c31|     c32|     c33|    c34|     c35|c36|    c37|     c38|     c39|  summary|
+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+---------+
|  0|  0|1559473|1559473|  0|5540625|  0|  0|  0|  0|  0|1559473|  0|  0|  0|1559473|  0|  0|20172858|20172858|1559473|34955073|  0|1559473|20172858|20172858|nan_count|
+---+---+-------+-------+---+-------+---+---+---+---+---+-------+---+---+---+-------+---+---+--------+--------+-------+--------+---+-------+--------+--------+---------+



In [127]:
stats2.union(newRow).show(vertical=True)

-RECORD 0------------
 summary | count     
 c14     | 45840617  
 c15     | 45840617  
 c16     | 44281144  
 c17     | 44281144  
 c18     | 45840617  
 c19     | 40299992  
 c20     | 45840617  
 c21     | 45840617  
 c22     | 45840617  
 c23     | 45840617  
 c24     | 45840617  
 c25     | 44281144  
 c26     | 45840617  
 c27     | 45840617  
 c28     | 45840617  
 c29     | 44281144  
 c30     | 45840617  
 c31     | 45840617  
 c32     | 25667759  
 c33     | 25667759  
 c34     | 44281144  
 c35     | 10885544  
 c36     | 45840617  
 c37     | 44281144  
 c38     | 25667759  
 c39     | 25667759  
-RECORD 1------------
 summary | 0         
 c14     | 0         
 c15     | 1559473   
 c16     | 1559473   
 c17     | 0         
 c18     | 5540625   
 c19     | 0         
 c20     | 0         
 c21     | 0         
 c22     | 0         
 c23     | 0         
 c24     | 1559473   
 c25     | 0         
 c26     | 0         
 c27     | 0         
 c28     | 1559473   
 c29     |

In [82]:
c = s.agg(F.collect_set('c16')).collect()

In [83]:
for i,v in enumerate(c[0][0][:5]):
    print(i, v)

0 b7cb86b6
1 ba124002
2 0ca4a150
3 2e0a5358
4 f8f74498


# Pre-processing

In [97]:
%%time
# get unique counts of categorical variables
uniqueCounts = []
for c in train.columns[14:]:
    count = train.agg(F.countDistinct(c).alias('c')).collect()[0]['c']
    uniqueCounts.append(count)
uniqueCounts = np.array(uniqueCounts)
print(uniqueCounts)

[   1609     600 9971778 1948105     343      24   11682     739       3
  102509    6817 7761967    3798      30   14555 4892269      11    6807
    2128       3 7830619      17      13  287894     107  135968]
CPU times: user 280 ms, sys: 100 ms, total: 380 ms
Wall time: 2min 47s


In [92]:
# wrong interpretation
# cols = train.columns[14:]
# for c, col in zip(uniqueCounts, cols):
#     if c < 4000000:
#         train = train.drop(col)
# train.columns

In [9]:
for i,c in enumerate(train.columns[1:14]):
    train = train.withColumn(c, (train[c] - means[i]) / stdevs[i])

In [10]:
stats2 = train[train.columns[1:14]].describe()
stats2.show(vertical=True)

-RECORD 0-----------------------
 summary | count                
 c1      | 5851479              
 c2      | 10517430             
 c3      | 8259890              
 c4      | 8205798              
 c5      | 10229986             
 c6      | 8113327              
 c7      | 10051997             
 c8      | 10511722             
 c9      | 10051997             
 c10     | 5851479              
 c11     | 10051997             
 c12     | 2421098              
 c13     | 8205798              
-RECORD 1-----------------------
 summary | mean                 
 c1      | -7.97593112269365... 
 c2      | 5.828207091676494... 
 c3      | 3.909513938232698... 
 c4      | -3.74083514832785... 
 c5      | 9.537483029736827... 
 c6      | 3.244443654866041... 
 c7      | 9.026366645182133... 
 c8      | -4.24857390976072... 
 c9      | 6.576557065533054... 
 c10     | 1.379353179119628... 
 c11     | 4.190010407219867... 
 c12     | -1.01941175954099... 
 c13     | -4.75671009901150... 
-RECORD 2-

# Experimenting with RDDs

In [16]:
toyTrainRDD = sc.textFile('data/toy_train.txt')

In [17]:
toyTrainRDD.collect()[0]

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [21]:
line = toyTrainRDD.collect()[0]
line

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [22]:
line = line.split('\t')
line

['0',
 '',
 '1',
 '12',
 '1',
 '62526',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '3',
 '05db9164',
 '08d6d899',
 '9143c832',
 'f56b7dd5',
 '25c83c98',
 '7e0ccccf',
 '89a13b6b',
 '0b153874',
 '7cc72ec2',
 'c5fe5cb9',
 'fa365cf9',
 'ae1bb660',
 'd2420e4c',
 'b28479f6',
 'bffbd637',
 'bad5ee18',
 '07c540c4',
 'bbf70d82',
 '',
 '',
 '0429f84b',
 '',
 '32c7478e',
 'c0d61a5c',
 '',
 '']

In [12]:
def parse(line):
    line = line.split('\t')
    key = line[0]
    values = line[1:]
    for i,v in enumerate(values[:13]):
        v = 
    
    return key, values

In [14]:
toyTrainRDD = toyTrainRDD.map(parse)

toyTrainRDD.collect()[0]

('0',
 ('',
  '1',
  '12',
  '1',
  '62526',
  '',
  '',
  '3',
  '',
  '',
  '',
  '',
  '3',
  '05db9164',
  '08d6d899',
  '9143c832',
  'f56b7dd5',
  '25c83c98',
  '7e0ccccf',
  '89a13b6b',
  '0b153874',
  '7cc72ec2',
  'c5fe5cb9',
  'fa365cf9',
  'ae1bb660',
  'd2420e4c',
  'b28479f6',
  'bffbd637',
  'bad5ee18',
  '07c540c4',
  'bbf70d82',
  '',
  '',
  '0429f84b',
  '',
  '32c7478e',
  'c0d61a5c',
  '',
  ''))